In [ ]:
!pip install datasets

In [ ]:
!pip install -U transformers

In [3]:
from IPython.display import display


In [14]:
import time
import psutil
import torch
from datasets import load_dataset
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments, DataCollatorWithPadding
import pandas as pd
from sklearn.metrics import accuracy_score

In [15]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(axis=-1)
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc}

In [5]:
model_name = "prajjwal1/bert-tiny"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
dataset = load_dataset("glue", "sst2")
dataset["train"].to_csv("sst2_train.csv", index=False)
dataset["validation"].to_csv("sst2_validation.csv", index=False)

Creating CSV from Arrow format:   0%|          | 0/68 [00:00<?, ?ba/s]

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating CSV from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

204804

In [8]:
dataset = load_dataset('csv', data_files={
    'train': 'sst2_train.csv',
    'validation': 'sst2_validation.csv'
})

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

In [9]:
def tokenize_function(example):
    return tokenizer(example['sentence'], truncation=True)
tokenized_datasets = dataset.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map:   0%|          | 0/67349 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/872 [00:00<?, ? examples/s]

In [12]:
import os
os.environ["WANDB_DISABLED"] = "true"


In [19]:
subset_sizes = [25, 50, 100]
results = []
for size in subset_sizes:
    train_dataset = tokenized_datasets['train'].shuffle(seed=42)
    n_total = len(train_dataset)
    subset = train_dataset.select(range(int(n_total * size / 100)))



    training_args = TrainingArguments(
        output_dir=f'./results_subset_{size}',
        num_train_epochs=5,
        per_device_train_batch_size=32,
        per_device_eval_batch_size=32,
        logging_steps=10,
        save_strategy="no",
        logging_strategy="no",
        disable_tqdm=False
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=subset,
        eval_dataset=tokenized_datasets['validation'],
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics
    )

    process = psutil.Process()
    mem_before = process.memory_info().rss / (1024 ** 2)
    start_time = time.time()

    train_output = trainer.train()
    eval_output = trainer.evaluate()

    time_elapsed = time.time() - start_time
    mem_after = process.memory_info().rss / (1024 ** 2)
    train_eval = trainer.evaluate(subset)

    results.append({
        'subset_size': size,
        'train_time_sec': round(time_elapsed, 2),
        'memory_before_mb': round(mem_before, 2),
        'memory_after_mb': round(mem_after, 2),
        'train_accuracy': round(train_eval.get('eval_accuracy', 0), 4),
        'eval_accuracy': round(eval_output.get('eval_accuracy', 0), 4)
    })



df_results = pd.DataFrame(results)
display(df_results)

df_results.to_csv("subset_selection_results.csv", index=False)


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
<ipython-input-19-bbcc60459bf0>:21: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
<ipython-input-19-bbcc60459bf0>:21: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
<ipython-input-19-bbcc60459bf0>:21: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss


,subset_size,train_time_sec,memory_before_mb,memory_after_mb,train_accuracy,eval_accuracy
0,25,37.42,1970.73,1972.31,0.9227,0.8005
1,50,68.49,1972.31,1973.07,0.9474,0.8211
2,100,138.05,1973.07,1974.11,0.9588,0.8177
